In [ ]:
import os
import random
import shutil
from pathlib import Path
import yaml
from ultralytics import YOLO

In [ ]:
base_path = Path("PATH_TO_AUGMENTED_DATA")
images_path = base_path / "images"
labels_path = base_path / "labels"

split_base = Path("PATH_TO_SPLIT_DATA")
for split in ["train", "val", "test"]:
    (split_base / split / "images").mkdir(parents=True, exist_ok=True)
    (split_base / split / "labels").mkdir(parents=True, exist_ok=True)

all_images = sorted(os.listdir(images_path))
random.shuffle(all_images)

train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1
n = len(all_images)
train_end = int(train_ratio * n)
val_end = int((train_ratio + val_ratio) * n)

train_files = all_images[:train_end]
val_files = all_images[train_end:val_end]
test_files = all_images[val_end:]

def move_files(file_list, split):
    for f in file_list:
        img_src = images_path / f
        label_src = labels_path / f.replace(".png", ".txt")
        img_dst = split_base / split / "images" / f
        label_dst = split_base / split / "labels" / f.replace(".png", ".txt")
        shutil.copy(img_src, img_dst)
        shutil.copy(label_src, label_dst)

move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print(f"Split completed: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

In [ ]:
model = YOLO("PATH_TO_YOLO_WEIGHTS")
train_results = model.train(
    data="PATH_TO_DATASET_YAML",
    epochs=100,
    imgsz=416,
    device="cuda",
    patience=10
    )

In [ ]:
metrics = model.val()

In [ ]:
results = model("PATH_TO_TEST_IMAGE")
results[0].show()

In [ ]:
model.export(format="onnx")